#### example of pyspark ml

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.appName('Missing').getOrCreate()

22/09/18 23:02:00 WARN Utils: Your hostname, MacBooks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.33 instead (on interface en0)
22/09/18 23:02:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 23:02:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/18 23:02:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/18 23:02:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
spark

In [8]:
#Read The dataset
training = spark.read.csv('/Users/macbookpro/Desktop/git/spark-exercise/test1_example.csv',header=True,inferSchema=True)

In [9]:
training.show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  mahsa| 31|        10| 30000|
|   sara| 30|         8| 25000|
| hanieh| 29|         4| 20000|
|  atefe| 24|         3| 20000|
|shabnam| 21|         1| 15000|
| bahare| 23|         2| 18000|
+-------+---+----------+------+



In [10]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [12]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [16]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [17]:
output=featureassembler.transform(training)
output.show()

+-------+---+----------+------+--------------------+
|   Name|age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|  mahsa| 31|        10| 30000|         [31.0,10.0]|
|   sara| 30|         8| 25000|          [30.0,8.0]|
| hanieh| 29|         4| 20000|          [29.0,4.0]|
|  atefe| 24|         3| 20000|          [24.0,3.0]|
|shabnam| 21|         1| 15000|          [21.0,1.0]|
| bahare| 23|         2| 18000|          [23.0,2.0]|
+-------+---+----------+------+--------------------+



In [18]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [19]:
finalized_data=output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

22/09/19 00:03:27 WARN Instrumentation: [3c6bcde8] regParam is zero, which might cause numerical instability and overfitting.
22/09/19 00:03:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/09/19 00:03:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/09/19 00:03:27 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [21]:
### Coefficients
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [22]:
### Intercepts
regressor.intercept

15414.10693970376

In [23]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [24]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18612.059158134223|
+--------------------+------+------------------+

